In [1]:
import os
import argparse
import numpy as np
import cv2
import time

In [2]:
avaliable = ['baggage_claim.jpg','dining_table.jpg','living_room.jpg','soccer.jpg']
image_directory = '/Users/sanjaymsanthosh/Desktop/projects/project 2019/human detection/yolo/yolo-object-detection/images/'
image_location = os.path.join(image_directory,avaliable[1])
yolo_directory = "/Users/sanjaymsanthosh/Desktop/projects/project 2019/human detection/yolo/yolo-object-detection/yolo-coco"

In [3]:
label_path = os.path.sep.join([yolo_directory,'coco.names'])
Labels = open(label_path).read().split('\n')


In [4]:
np.random.seed(42)
Colors = np.random.randint(0,255,[len(Labels),3])

confidence_score = .5
threshold_score = .3

In [5]:
weightsPath = os.path.sep.join([yolo_directory, "yolov3.weights"])
configPath = os.path.sep.join([yolo_directory, "yolov3.cfg"])

In [6]:
print('[INFO] loading yolo pre trained from disk')
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

[INFO] loading yolo pre trained from disk


In [7]:
images = cv2.imread(image_location)
(H,W)=images.shape[:2]

In [8]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [9]:
blob = cv2.dnn.blobFromImage(images, 1 / 255.0, (416, 416),swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()

In [10]:
# show timing information on YOLO
print("[INFO] YOLO took {:.6f} seconds".format(end - start))

# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively
boxes = []
confidences = []
classIDs = []

[INFO] YOLO took 0.678042 seconds


In [11]:
for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
        # extract the class ID and confidence (i.e., probability) of
        # the current object detection
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]

        # filter out weak predictions by ensuring the detected
        # probability is greater than the minimum probability
        if confidence > confidence_score:
            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the boundin
            # box followed by the boxes' width and height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")

            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)



In [12]:
idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_score,threshold_score)

In [13]:
if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
        # extract the bounding box coordinates
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        # draw a bounding box rectangle and label on the image
        color = [int(c) for c in Colors[classIDs[i]]]
        cv2.rectangle(images, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(Labels[classIDs[i]], confidences[i])
        cv2.putText(images, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)

In [ ]:
cv2.imshow("Image", images)
cv2.waitKey(0)